# Setting Up the Data (OMOP-STARR) 
Based off of work from: https://github.com/clinicalml/omop-learn

#### Imports & STARR OMOP Setups

In [1]:
import sys
import time
import importlib
import sparse
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt

In [2]:
##Setting up Google sdk environment
import os 

#Credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = f'/Users/{os.getlogin()}/.config/gcloud/application_default_credentials.json' 

#Projects & Data
dataset_id = 'som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest'
project_id = 'som-nero-nigam-bmi215'

#Nero Project
os.environ['GCLOUD_PROJECT'] = project_id

##Setting up the python environment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

##Setting up BQ API
from google.cloud import bigquery
client=bigquery.Client()

/Users/cecileloge/opt/anaconda3/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## 1. Build a Cohort for the End of Life Prediction Task

In [ ]:
eol_dataset_id = 'end_of_life_task'

# Creating the dataset on GCP - Comment out if it already exists
client.create_dataset(f'{project_id}.{eol_dataset_id}')

In [14]:
#Setup for the EoL Cohort pull
schema_name = eol_dataset_id
omop_schema = dataset_id
training_start_date = '2016-01-01' #start date observation
training_end_date = '2017-01-01' #end date for observation (start + 1year)
start_window = '2017-04-01' #end date + 3months i.e the outcome window starts 3 months after the end of the obs. period
end_window = '2017-10-01' #end date + gap + 6months
days_gap_outcome = 273.75  #number of days in gap + outcome window (i.e. 3 + 6 = 9months)

In [15]:
#Construct the End of Life cohort table
#Inclusion criteria (matching SARD):
#    - Enrolled in 95% of months of training
#    - Enrolled in 95% of days during outcome window (test), or expired during outcome window
#    - Patient over the age of 70 at prediction time
#    - Outcome is "Death"

sql = """
create or replace table {schema_name}.{table_name} as

with
    death_dates as (
        select
            person_id,
            death_date
        from
            {omop_schema}.death 
    ),
    eligible_people as (
        select person_id
        from {omop_schema}.person 
        where extract(
            year from date '{training_end_date}'
        ) - year_of_birth > 70
    ),
    death_training_elig_counts as (
        select
            o.person_id AS person_id,
            o.observation_period_start_date as start,
            o.observation_period_end_date as finish,
            greatest(date_diff(
                least (
                    o.observation_period_end_date,
                    date '{training_end_date}'
                ), greatest(
                    o.observation_period_start_date,
                    date '{training_start_date}'
                ), DAY), 0
            ) as num_days
        from {omop_schema}.observation_period o
        inner join eligible_people p
        on o.person_id = p.person_id
    ),
    death_trainingwindow_elig_perc as (
        select
            person_id
        from
            death_training_elig_counts
        group by
            person_id
        having
            sum(num_days) >= 0.95 * (date_diff(date '{training_end_date}',date '{training_start_date}', DAY))
    ),
    death_testperiod_elig_counts as (
        select
            p.person_id,
            p.observation_period_start_date as start,
            p.observation_period_end_date as finish,
            greatest(
                    date_diff(least (
                        p.observation_period_end_date,
                        date '{limit_date}'
                    ), greatest(
                        p.observation_period_start_date,
                        date '{training_end_date}'
                    ), DAY), 0
            ) as num_days
        from {omop_schema}.observation_period p
        inner join 
            death_trainingwindow_elig_perc tr
        on 
            tr.person_id = p.person_id
    ), 
    death_testwindow_elig_perc as (
        select
            dtec.person_id as person_id
        from
            death_testperiod_elig_counts dtec
        join 
            death_dates d  
        on 
            dtec.person_id = d.person_id
        group by 
            dtec.person_id, d.death_date  
        having
            (d.death_date >= date '{mid_limit_date}' and
             d.death_date <= date '{limit_date}') 
        or
            sum(num_days) >= 0.95 * {days_gap_outcome}
    ) 
    
    select
        row_number() over (order by te.person_id) - 1 as example_id,
        te.person_id,
        date '{training_start_date}' as start_date,
        date '{training_end_date}' as end_date,
        d.death_date as outcome_date,
        
        cast(coalesce(
            (d.death_date between
                date '{mid_limit_date}'
                and
                date '{limit_date}'
            ), false
        ) as INT64) as y
    from
        death_testwindow_elig_perc te
        left join death_dates d on d.person_id = te.person_id
    where
        (
            d.death_date is null
            or d.death_date >= (date '{mid_limit_date}')
        )
    ;
 """.format(
    schema_name = schema_name,
    table_name = '__eol_cohort',
    omop_schema = omop_schema,
    training_start_date = training_start_date,
    training_end_date = training_end_date,
    mid_limit_date = start_window,
    limit_date = end_window,
    days_gap_outcome = days_gap_outcome
)
query_job = client.query(sql)

## 2. Build a Cohort for the Surgical Procedure Prediction Task

In [ ]:
surg_dataset_id = 'surgical_task'

# Creating the dataset on GCP - Comment out if it already exists
client.create_dataset(f'{project_id}.{surg_dataset_id}')

In [5]:
#Setup for the Surgical Procedure Cohort pull
schema_name = surg_dataset_id
omop_schema = dataset_id
training_start_date = '2016-01-01' #start date observation
training_end_date = '2017-01-01' #end date for observation (start + 1year)
start_window = '2017-04-01' #end date + 3months i.e the outcome window starts 3 months after the end of the obs. period
end_window = '2017-10-01' #end date + gap + 6months
days_gap_outcome = 273.75  #number of days in gap + outcome window (i.e. 3 + 6 = 9months)

In [6]:
#Construct the Surgical Procedure cohort table
#Inclusion criteria
#    - Enrolled in 95% of months of training
#    - Enrolled in 95% of days during outcome window (test)
#    - Outcome is "Surgical Procedure"

sql = """
create or replace table {schema_name}.{table_name} as

with
    surg_dates as (
        select
            person_id,
            procedure_date
        from
            {omop_schema}.procedure_occurrence
        where procedure_concept_id IN (select concept_id from {omop_schema}.concept
                                       WHERE domain_id = 'Procedure' and standard_concept = 'S' and concept_name LIKE '%surgical%' )
              and procedure_date >= (date '{mid_limit_date}')
    ),
    eligible_people as (
        select person_id
        from {omop_schema}.person 
    ),
    training_elig_counts as (
        select
            o.person_id AS person_id,
            o.observation_period_start_date as start,
            o.observation_period_end_date as finish,
            greatest(date_diff(
                least (
                    o.observation_period_end_date,
                    date '{training_end_date}'
                ), greatest(
                    o.observation_period_start_date,
                    date '{training_start_date}'
                ), DAY), 0
            ) as num_days
        from {omop_schema}.observation_period o
        inner join eligible_people p
        on o.person_id = p.person_id
    ),
    trainingwindow_elig_perc as (
        select
            person_id
        from
            training_elig_counts
        group by
            person_id
        having
            sum(num_days) >= 0.95 * (date_diff(date '{training_end_date}',date '{training_start_date}', DAY))
    ),
    testperiod_elig_counts as (
        select
            p.person_id,
            p.observation_period_start_date as start,
            p.observation_period_end_date as finish,
            greatest(
                    date_diff(least (
                        p.observation_period_end_date,
                        date '{limit_date}'
                    ), greatest(
                        p.observation_period_start_date,
                        date '{training_end_date}'
                    ), DAY), 0
            ) as num_days
        from {omop_schema}.observation_period p
        inner join 
            trainingwindow_elig_perc tr
        on 
            tr.person_id = p.person_id
    ), 
    testwindow_elig_perc as (
        select
            dtec.person_id as person_id
        from
            testperiod_elig_counts dtec
        group by 
            dtec.person_id  
        having
            sum(num_days) >= 0.95 * {days_gap_outcome}
    ) 
    
    select
        row_number() over (order by te.person_id) - 1 as example_id,
        te.person_id,
        date '{training_start_date}' as start_date,
        date '{training_end_date}' as end_date,
        min(d.procedure_date) as outcome_date
    from
        testwindow_elig_perc te
        left join surg_dates d on d.person_id = te.person_id
    where
        (
            d.procedure_date is null
            or d.procedure_date >= (date '{mid_limit_date}')
        )
    group by 2,3,4
    ;
 """.format(
    schema_name = schema_name,
    table_name = '__surg_cohort',
    omop_schema = omop_schema,
    training_start_date = training_start_date,
    training_end_date = training_end_date,
    mid_limit_date = start_window,
    limit_date = end_window,
    days_gap_outcome = days_gap_outcome
)
query_job = client.query(sql)

In [18]:
#Adding y

sql = """
create or replace table {schema_name}.{table_name} as
select 
    *,
    cast(coalesce(
            (outcome_date between
                date '{mid_limit_date}'
                and
                date '{limit_date}'
            ), false
        ) as INT64) as y
from 
    {project_id}.{schema_name}.{table_name} 

 """.format(
    project_id = project_id,
    schema_name = schema_name,
    table_name = '__surg_cohort',
    mid_limit_date = start_window,
    limit_date = end_window
)
query_job = client.query(sql)

In [20]:
# Checking class imbalance
sql = """
select 
    sum(y) as y,
    count(y) as allcount
from 
    {project_id}.{schema_name}.{table_name} 

 """.format(
    project_id = project_id,
    schema_name = schema_name,
    table_name = '__surg_cohort',
    mid_limit_date = start_window,
    limit_date = end_window
)
query_job = client.query(sql)
test = query_job.to_dataframe()
test

,y,allcount
0,17702,790167


## 3. Set Up the Features... Drugs, Conditions, Procedures, Specialty

In [9]:
#Setup for the Feature Pull
schema_name = surg_dataset_id # can be surg_dataset_id or eol_dataset_id
cohort_table_name = '__surg_cohort' # can be '__surg_cohort' or '__eol_cohort'
drugs_name = 'drugs'
conditions_name = 'conditions'
proc_name = 'procedures'
specialty_name = 'specialty'
omop_schema = dataset_id

In [10]:
#DRUGS FEATURES

sql = """
create or replace table {schema_name}.{table_name} as
select 
    b.example_id,
    a.person_id,
    a.drug_concept_id as concept_id,
    a.drug_concept_id || ' - drug - ' || coalesce (
        c.concept_name, 'no match'
    ) as concept_name,
    a.drug_exposure_start_date as feature_start_date,
    b.start_date as person_start_date,
    b.end_date as person_end_date,
    date_diff(b.end_date, a.drug_exposure_start_date, DAY) as number_days_from_end,
    
from 
    {omop_schema}.drug_exposure a
inner join
    {project_id}.{schema_name}.{cohort_table} b
on 
    a.person_id = b.person_id
left join
    {omop_schema}.concept c
on 
    c.concept_id = a.drug_concept_id
where date_diff(b.end_date, a.drug_exposure_start_date, DAY) >= 0
 """.format(
    schema_name = schema_name,
    table_name = drugs_name,
    cohort_table = cohort_table_name,
    omop_schema = omop_schema,
    project_id = project_id
)
query_job = client.query(sql)

In [11]:
#CONDITIONS FEATURES

sql = """
create or replace table {schema_name}.{table_name} as
select 
    b.example_id,
    a.person_id,
    a.condition_concept_id as concept_id,
    a.condition_concept_id || ' - condition - ' || coalesce (
        c.concept_name, 'no match'
    ) as concept_name,
    a.condition_start_date as feature_start_date,
    b.start_date as person_start_date,
    b.end_date as person_end_date,
    date_diff(b.end_date, a.condition_start_date, DAY) as number_days_from_end
from 
    {omop_schema}.condition_occurrence a
inner join
    {project_id}.{schema_name}.{cohort_table} b
on 
    a.person_id = b.person_id
left join
    {omop_schema}.concept c
on 
    c.concept_id = a.condition_concept_id
where date_diff(b.end_date, a.condition_start_date, DAY) >= 0
 """.format(
    schema_name = schema_name,
    table_name = conditions_name,
    cohort_table = cohort_table_name,
    omop_schema = omop_schema,
    project_id = project_id
)
query_job = client.query(sql)

In [12]:
#PROCEDURES FEATURES

sql = """
create or replace table {schema_name}.{table_name} as
select 
    b.example_id,
    a.person_id,
    a.procedure_concept_id as concept_id,
    a.procedure_concept_id || ' - procedure - ' || coalesce (
        c.concept_name, 'no match'
    ) as concept_name,
    a.procedure_date as feature_start_date,
    b.start_date as person_start_date,
    b.end_date as person_end_date,
    date_diff(b.end_date, a.procedure_date, DAY) as number_days_from_end
from 
    {omop_schema}.procedure_occurrence a
inner join
    {project_id}.{schema_name}.{cohort_table} b
on 
    a.person_id = b.person_id
left join
    {omop_schema}.concept c
on 
    c.concept_id = a.procedure_concept_id
where date_diff(b.end_date, a.procedure_date, DAY) >= 0
 """.format(
    schema_name = schema_name,
    table_name = proc_name,
    cohort_table = cohort_table_name,
    omop_schema = omop_schema,
    project_id = project_id
)
query_job = client.query(sql)

In [13]:
#SPECIALTY FEATURES

sql = """
create or replace table {schema_name}.{table_name} as
select 
    b.example_id,
    a.person_id,
    p.specialty_concept_id as concept_id,
    p.specialty_concept_id || ' - specialty - ' || coalesce (
        c.concept_name, 'no match'
    ) as concept_name,
    a.visit_start_date as feature_start_date,
    b.start_date as person_start_date,
    b.end_date as person_end_date,
    date_diff(b.end_date, a.visit_start_date, DAY) as number_days_from_end 
from 
    {omop_schema}.visit_occurrence a
inner join
    {project_id}.{schema_name}.{cohort_table} b
on 
    a.person_id = b.person_id
inner join
    {omop_schema}.provider p
on
     a.provider_id = p.provider_id
left join
    {omop_schema}.concept c
on 
    c.concept_id = p.specialty_concept_id
where date_diff(b.end_date, a.visit_start_date, DAY) >= 0
 """.format(
    schema_name = schema_name,
    table_name = specialty_name,
    cohort_table = cohort_table_name,
    omop_schema = omop_schema,
    project_id = project_id
)
query_job = client.query(sql)

In [16]:
# Unioning them all into one big feature table

sql = """
create or replace table {schema_name}.{table_name} as
select * 
from 
{project_id}.{schema_name}.{drugs_name}

union all

select *
from
{project_id}.{schema_name}.{conditions_name}

union all

select *
from
{project_id}.{schema_name}.{proc_name}

union all

select * 
from
{project_id}.{schema_name}.{specialty_name}
 """.format(
    schema_name = schema_name,
    table_name = 'all_features',
    omop_schema = omop_schema,
    project_id = project_id,
    drugs_name = drugs_name,
    conditions_name = conditions_name,
    proc_name = proc_name,
    specialty_name = specialty_name
)
query_job = client.query(sql)


#### Adding the Visit number

In [17]:
#ALL FEATURES / RANKING VISITS

sql = """
create or replace table {schema_name}.{table_name} as
select 
    *,
    DENSE_RANK() OVER (PARTITION BY person_id ORDER BY feature_start_date ASC) AS visit_number
from 
    {project_id}.{schema_name}.{table_name} 

where concept_id != 0

 """.format(
    project_id = project_id,
    schema_name = schema_name,
    table_name = 'all_features'
)
query_job = client.query(sql)